In [2]:
import os
import h5py
import pickle
import numpy as np
import napari
from datetime import date
from skimage import morphology
from skimage.filters import gaussian
from skimage.exposure import rescale_intensity
from skimage.transform import resize
from pystackreg import StackReg
from scipy import ndimage as ndi
from scipy.stats import zscore

In [3]:
def matrix_rc2xy(affine_matrix):
    swapped_cols = affine_matrix[:, [1, 0, 2]]
    swapped_rows = swapped_cols[[1, 0, 2], :]
    return swapped_rows


def find_translation(source,destination):
    
    sr = StackReg(StackReg.TRANSLATION)
    tmat = sr.register(destination,source)
    
    tmat = matrix_rc2xy(tmat)
    
    return tmat

def find_scaled_rotation(source,destination):
    
    sr = StackReg(StackReg.SCALED_ROTATION)
    tmat = sr.register(destination,source)
    
    tmat = matrix_rc2xy(tmat)

    return tmat

In [4]:
# check date

today = date.today()
today = today.strftime("%Y%m%d")
today

'20220815'

In [9]:
im_path = r'W:\Users\kmkedz\Dragonfly PSFs\Single cam example\PSF_63X_1.0X_WF_Zyla_2022-02-17_09.55.03.ims' 

save_dir = r'W:\Users\kmkedz\Dragonfly PSFs'
save_file = f'{today}_3D_zyla_1x_63x_fr_r_g_b' #f'{today}_MSL_zyla_20x'
save_file

'20220815_3D_zyla_1x_63x_fr_r_g_b'

In [6]:
channel_names = ['ch_637','ch_561','ch_488','ch_405']
vis_colors = ['magenta','red','green','blue'] #['green','magenta']#

z_step = 99 #[nm]

ch_num = len(channel_names)

## Read handles to stacks

In [7]:
f = h5py.File(im_path,'r')

In [8]:
for i in range(ch_num):

    stack = f['DataSet']['ResolutionLevel 0']['TimePoint 0'][f'Channel {i}']['Data']
    exec("stack_"+channel_names[i]+"= stack")
    
if (len(stack.shape) > 2 and (stack.shape[0] > 1)):
    
    print(f'Dimensions of the selected stack: {stack.shape}')

else:
    
    print(f'Warning - selected data set is 2D!!!!')

Dimensions of the selected stack: (128, 2048, 2048)


## Translation in z direction

In [107]:
# make projection in zy
# this may take a few minutes

pos = 100
rad = 20

sigma = 3
# how much is cut from the top and the bottom of yz projection
margin = 10

zy_list = []
for i in range(ch_num):
    
    exec("stack = stack_"+channel_names[i])
    
    zy = np.max(stack[margin:-margin,pos-rad:pos+rad,:],axis=1)
    
    zy = gaussian(zy, sigma,preserve_range=True)
    
    zy = zscore(zy,axis=None)
    
    zy_list.append(zy)
    
    exec("zy_"+channel_names[i]+"= zy")

In [108]:
# visualize original

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = zy_"+channel_names[i])
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],
                    name = channel_names[i])

C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [109]:
# alignement in yz plane

destination = zy_list[0]

tmat_z_list = []

for i in range(ch_num):
    
    exec("source = zy_"+channel_names[i])
    
    tmat = find_translation(source,destination)
    
    ch_aligned = ndi.affine_transform(source, tmat)
    
    exec("zy_"+channel_names[i]+"_aligned = ch_aligned")
    
    tmat_z_list.append(tmat)

In [110]:
# check the alignment in z

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = zy_"+channel_names[i]+"_aligned")
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],
                    name = channel_names[i])

C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


## Alignment in xy

In [111]:
# make projection in xy

pos = 50
rad = 30

for i in range(ch_num):
    
    exec("stack = stack_"+channel_names[i])
    
    xy = np.max(stack[pos-rad:pos+rad,:,:],axis=0)
    
    exec("xy_"+channel_names[i]+"= xy")

In [112]:
# processing of xy images

sigma = 2
footprint = morphology.disk(3)

xy_list = []

for i in range(ch_num):
    
    exec("xy = xy_"+channel_names[i])
    
    xy = morphology.white_tophat(xy, footprint)
    xy = gaussian(xy, sigma)
    
    # normalize
    #xy = xy/np.percentile(xy,99.999)
    xy = zscore(xy,axis=None)
    
    xy_list.append(xy)
    
    exec("xy_"+channel_names[i]+"= xy")

In [113]:
# calculate reference image

#xy_ref = np.mean(np.array(xy_list),axis=0)
#xy_ref = xy_ref/np.percentile(xy_ref,99.999)

In [114]:
# visualize original xy data

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = xy_"+channel_names[i])
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],name = channel_names[i])

C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [115]:
# alignment in xy plane

destination = xy_list[0]

tmat_xy_list = []

for i in range(ch_num):
    
    exec("source = xy_"+channel_names[i])
    
    tmat = find_scaled_rotation(source,destination)
    
    ch_aligned = ndi.affine_transform(source, tmat)
    
    exec("xy_"+channel_names[i]+"_aligned = ch_aligned")
    
    tmat_xy_list.append(tmat)


In [116]:
# check the alignment in xy

s_fact = 99.99

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = xy_"+channel_names[i]+"_aligned")
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],contrast_limits=(0,np.percentile(im,s_fact)),
                    name = channel_names[i])

C:\Users\kmkedz\.conda\envs\napari-tests\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


## Combine into 2.5D transformation

In [117]:
# create transformations 3D

tmat_xyz = []

for tmat_xy,tmat_z in zip(tmat_xy_list,tmat_z_list):
   
    tmat = np.zeros([4,4])
    
    tmat[:2,:2] = tmat_xy[:2,:2]
    tmat[:2,3] = tmat_xy[:2,2]
    tmat[2,2] = 1
    tmat[2,3] = tmat_z[0,2]
    tmat[3,3] = 1
    
    tmat_xyz.append(tmat)

## Save transformations

In [118]:
with open(os.path.join(save_dir,save_file+'.pkl'),"wb") as f:
    pickle.dump([channel_names,tmat_z_list,tmat_xy_list,tmat_xyz,z_step],f)


with open(os.path.join(save_dir,save_file+'.txt'), "w") as f:
    
    f.write(f'Alignment calculated {today}.')
    f.write('\n')
    f.write('\n')
    
    f.write(f'Channels:')
    f.write('\n')
    f.write('\n')

    for tmat in channel_names:
        f.write(tmat)
        f.write('\n')
    
    f.write('\n')
    f.write(f'Alignment in z:')
    f.write('\n')
    f.write('\n')

    for tmat in tmat_z_list:
        np.savetxt(f, tmat, delimiter=',')
        f.write('\n')
        
    f.write('\n')
    f.write(f'Step in z:')
    f.write('\n')
    f.write('\n')
    f.write(str(z_step))
    f.write('\n')

        
    f.write('\n')
    f.write(f'Alignment in xy:')
    f.write('\n')
    f.write('\n')

    for tmat in tmat_xy_list:
        np.savetxt(f, tmat, delimiter=',')
        f.write('\n')
        
    f.write('\n')
    f.write(f'Alignment in xyz (2.5D):')
    f.write('\n')
    f.write('\n')

    for tmat in tmat_xyz:
        np.savetxt(f, tmat, delimiter=',')
        f.write('\n')